In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import altair as alt
import seaborn as sns
import numpy as np

### 1. load cross reference data and group by LSOA11CD

In [ ]:
cross_ref = pd.read_csv("Output_Area_to_LSOA_to_MSOA_to_Local_Authority_District_(December_2017)_Lookup_with_Area_Classifications_in_Great_Britain.csv")

cross_ref.drop(['OA11CD'], axis=1, inplace=True)
cross_ref.drop(['LAD17CD'], axis=1, inplace=True)

In [ ]:
lsoa_to_msoa = cross_ref.groupby(['LSOA11CD','LSOA11NM','MSOA11CD','MSOA11NM']).count().reset_index()
lsoa_to_msoa.count()

### 2. load london only covid data and group it all into one table
From this one dataset (containing data up to 28 May, 2020) we load and then merge the following:
- deaths 
- population over 70
- ethnic_group
- medical_conditions

In [ ]:
deaths = pd.read_excel("underlying_data_2020_06_01.xlsx", sheet_name='1 deaths')
population = pd.read_excel("underlying_data_2020_06_01.xlsx", sheet_name='2 population')
ethnicity = pd.read_excel("underlying_data_2020_06_01.xlsx", sheet_name='4 ethnic_group')
health = pd.read_excel("underlying_data_2020_06_01.xlsx", sheet_name='7 medical_conditions')

london_covid_total = pd.merge(deaths, population, left_on='MSOA11CD', right_on='MSOA11CD', how = 'inner')
london_covid_total = pd.merge(london_covid_total, ethnicity, left_on='MSOA11CD', right_on='MSOA11CD', how = 'inner')
london_covid_total = pd.merge(london_covid_total, health, left_on='MSOA11CD', right_on='MSOA11CD', how = 'inner')

london_covid_total.count()

In [ ]:
london_covid_total.drop(['MSOA11NM_x'], axis=1, inplace=True)
london_covid_total.drop(['MSOA11NM_y'], axis=1, inplace=True)
london_covid_total.drop(['Local authority_x'], axis=1, inplace=True)
london_covid_total.drop(['Local authority_y'], axis=1, inplace=True)
london_covid_total.drop(['Local Authority'], axis=1, inplace=True)
london_covid_total.head()

### 3. load deprivation data
The domains of deprivation gives us a ranking from 1 to many thousands for deprivation along a number of dimensions:
- income_rank
- environment_rank
- education_rank
- health_rank
- housing_rank
- living_environment_rank

In [ ]:
uk_deprivation_data = pd.read_excel("File_2_-_IoD2019_Domains_of_Deprivation.xlsx", sheet_name='IoD2019 Domains',
                                    usecols = "A,B,C,D,G,I,K, M, Q, S", names=['lsoa_code', 'lsoa_name', 
                                                            'local_auth_code', 'local_auth_name',  'income_rank', 
                                                            'employment_rank', 'education_rank', 'health_rank',
                                                            'housing_rank', 'living_environment_rank'])

london_deprivation = uk_deprivation_data[uk_deprivation_data['local_auth_code'].str.contains('E09')]
print(london_deprivation.count())
london_deprivation.head()

### Now merge deprivation and cross reference to get the link between lsoa and msoa

In [ ]:
# Now merge with cross ref and then group by msoa code

print(london_deprivation.count())
london_deprivation_new = pd.merge(london_deprivation, lsoa_to_msoa, left_on='lsoa_code', right_on='LSOA11CD', how = 'inner')

print(london_deprivation_new.count())

london_deprivation_new.head()

### Now group by MSOA and then merge with London covid data to get one large dataset

In [ ]:
london_deprivation_msoa = london_deprivation_new.groupby(['local_auth_code', 
                                                          'local_auth_name', 'MSOA11CD', 'MSOA11NM']).median().reset_index()

print(london_deprivation_msoa.count())
london_deprivation_msoa.head()

### Now merge with the covid deaths data based on msoa code

In [ ]:
london_covid_all = pd.merge(london_covid_total, london_deprivation_msoa, 
                            left_on='MSOA11CD', right_on='MSOA11CD', how = 'inner')

print(london_covid_all.count())
london_covid_all.head()

In [ ]:
london_covid_all.drop(['Local authority'], axis=1, inplace=True)
london_covid_all.drop(['total_registered_patients'], axis=1, inplace=True)

london_covid_all.rename(columns = {'MSOA11CD':'msoa_code'}, inplace = True)
london_covid_all.rename(columns = {'MSOA11NM':'msoa_name'}, inplace = True)
london_covid_all.rename(columns = {'Hypertension':'hypertension'}, inplace = True)
london_covid_all.rename(columns = {'Obesity (18+)':'obesity'}, inplace = True)
london_covid_all.rename(columns = {'Diabetes':'diabetes'}, inplace = True)
london_covid_all.rename(columns = {'Asthma':'asthma'}, inplace = True)
london_covid_all.rename(columns = {'Coronary heart disease':'heart_disease'}, inplace = True)

london_covid_all.head()


### ONS monthly death data
The London death data only extends to 17th April and so doesn't really encompass the entire first wave, and for that we would need data extending to the end of June, 2020. 
- We therefore load additional ONS monthly death data from March to end of July to provide a more comprehensive set of death data that covers the vast majority of deaths within the first wave. THis data is also at MSOA level

In [ ]:
ONS_covid_deaths = pd.read_excel("covidlocalareadeprivationupdate.xlsx", sheet_name='Table 5', 
                                 usecols = "A,Q:U", names=['msoa_code', 'march_deaths', 
                                                           'april_deaths', 'may_deaths', 'june_deaths', 'july_deaths'], skiprows=12)

ONS_covid_deaths['ons_total_deaths'] = ONS_covid_deaths.march_deaths + ONS_covid_deaths.april_deaths + ONS_covid_deaths.may_deaths + ONS_covid_deaths.june_deaths + ONS_covid_deaths.july_deaths

print(ONS_covid_deaths.count())
ONS_covid_deaths.head()

#### Merge with london_covid_all

In [ ]:
london_covid_all_ons = pd.merge(london_covid_all, ONS_covid_deaths, 
                                left_on='msoa_code', right_on='msoa_code', how = 'inner')

print(london_covid_all_ons.count())
london_covid_all_ons.head(10)

In [ ]:
# first add a field for ons deaths as a proportion of population and then reorder
london_covid_all_ons['ons_deaths_per_thousand'] = (1000*london_covid_all_ons.ons_total_deaths) / london_covid_all_ons.total_population_mid_2018
# now reorder before melting
london_covid_all_ons = london_covid_all_ons[['msoa_code', 'msoa_name', 'local_auth_code', 'local_auth_name', 
                                     'covid_19_deaths', 'covid_19_deaths_per_thousand', 'ons_total_deaths', 
                                     'ons_deaths_per_thousand', 'march_deaths', 'april_deaths', 'may_deaths',
                                     'june_deaths', 'july_deaths', 'total_population_mid_2018',
                                     'over_70_prop', 'all_bame_prop', 'all_black_prop', 'pakistani_or_bangladeshi_prop',
                                     'all_indian_prop', 'hypertension',
                                     'obesity', 'diabetes', 'asthma', 'heart_disease', 'income_rank', 
                                     'education_rank', 'health_rank', 'housing_rank', 'living_environment_rank'
                                    ]]

print(london_covid_all_ons.count())
london_covid_all_ons.head()

In [ ]:
col_names = london_covid_all_ons.columns.tolist()
col_names = col_names[8:29]

london_covid_stats = pd.melt(london_covid_all_ons, id_vars=['msoa_code', 
                                                        'msoa_name', 
                                                        'local_auth_code', 
                                                        'local_auth_name',
                                                        'covid_19_deaths', 
                                                        'covid_19_deaths_per_thousand',
                                                        'ons_total_deaths',
                                                        'ons_deaths_per_thousand'
                                                       ], 
                        value_vars=col_names, var_name = 'measure', value_name='value')

print(london_covid_stats.count())
london_covid_stats.head()

In [ ]:
london_covid_all_ons.to_csv('output_data/london_covid_all_ons.csv', index=False)

### Now we create groups of similar stats that we can compare with covid death rate to identify correlations
We we will create 3 groups:
- age and ethnicity profile
- health profile
- deprivation profile

- potential field values
msoa_code	msoa_name	local_auth_code	local_auth_name	covid_19_deaths	covid_19_deaths_per_thousand	total_population_mid_2018	over_70_prop	all_bame_prop	all_black_prop	pakistani_or_bangladeshi_prop	all_indian_prop	hypertension	obesity	diabetes	asthma	heart_disease	income_rank	environment_rank	education_rank	health_rank	housing_rank	living_environment_rank

In [ ]:
london_demographic = london_covid_stats[['msoa_code', 
                                       'msoa_name',
                                        'measure',
                                        'value',
                                        'covid_19_deaths',
                                        'covid_19_deaths_per_thousand',
                                        'ons_total_deaths',
                                        'ons_deaths_per_thousand']][london_covid_stats.measure.isin(['',
                                                                               'over_70_prop', 
                                                                               'all_black_prop', 
                                                                               'pakistani_or_bangladeshi_prop', 
                                                                               'all_indian_prop'])].copy().reset_index()

london_demographic.head()

In [ ]:
import altair as alt

alt.Chart(london_demographic).mark_point().encode(
    x='covid_19_deaths_per_thousand:Q',
    y='value:Q', 
    tooltip = ['msoa_name']
).properties(
    width=180,
    height=180
).facet(
    column='measure:N'
).interactive()

In [ ]:
import altair as alt

alt.Chart(london_demographic).mark_point().encode(
    x='ons_deaths_per_thousand:Q',
    y='value:Q', 
    tooltip = ['msoa_name']
).properties(
    width=180,
    height=180
).facet(
    column='measure:N'
).interactive()

In [ ]:
london_health = london_covid_stats[['msoa_code', 
                                       'msoa_name',
                                        'measure',
                                        'value',
                                        'covid_19_deaths',
                                        'covid_19_deaths_per_thousand',
                                        'ons_total_deaths',
                                        'ons_deaths_per_thousand']][london_covid_stats.measure.isin(['hypertension',
                                                                               'obesity', 
                                                                               'diabetes',
                                                                               'heart_disease',
                                                                               'asthma'])].copy().reset_index()

london_health.head()

In [ ]:
import altair as alt

alt.Chart(london_health).mark_point().encode(
    x='covid_19_deaths_per_thousand:Q',
    y='value:Q', 
    tooltip = ['msoa_name']
).properties(
    width=180,
    height=180
).facet(
    column='measure:N'
).interactive()

In [ ]:
import altair as alt

alt.Chart(london_health).mark_point().encode(
    x='ons_deaths_per_thousand:Q',
    y='value:Q', 
    tooltip = ['msoa_name']
).properties(
    width=180,
    height=180
).facet(
    column='measure:N'
).interactive()

In [ ]:
london_deprivation = london_covid_stats[['msoa_code', 
                                       'msoa_name',
                                        'measure',
                                        'value',
                                        'covid_19_deaths',
                                        'covid_19_deaths_per_thousand',
                                        'ons_total_deaths',
                                        'ons_deaths_per_thousand']][london_covid_stats.measure.isin(['income_rank',
                                                                               'environment_rank', 
                                                                               'education_rank',
                                                                               'housing_rank',
                                                                               'living_environment_rank'])].copy().reset_index()

london_health.head()

In [ ]:
import altair as alt

alt.Chart(london_deprivation).mark_point().encode(
    x='covid_19_deaths_per_thousand:Q',
    y='value:Q', 
    tooltip = ['msoa_name']
).properties(
    width=180,
    height=180
).facet(
    column='measure:N'
).interactive()

In [ ]:
import altair as alt

alt.Chart(london_deprivation).mark_point().encode(
    x='ons_deaths_per_thousand:Q',
    y='value:Q', 
    tooltip = ['msoa_name']
).properties(
    width=180,
    height=180
).facet(
    column='measure:N'
).interactive()

### No obvious correlations 
There aren't any obvious correlations between deaths per thousand and the demographic, health or deprivation profiles in the MSOA levels. Let's confirm this by formally checking the correlations

In [ ]:
london_corr_input = london_covid_all_ons[['covid_19_deaths_per_thousand', 'total_population_mid_2018',
                                     'over_70_prop', 'all_bame_prop', 'all_black_prop', 'pakistani_or_bangladeshi_prop',
                                     'all_indian_prop', 'hypertension',
                                     'obesity', 'diabetes', 'asthma', 'heart_disease', 'income_rank', 
                                     'education_rank', 'health_rank', 'housing_rank', 'living_environment_rank']]
london_corr = london_corr_input.corr()

corr_deaths=london_corr[['covid_19_deaths_per_thousand']]
#sort by the amount of correlation
corr_deaths=corr_deaths.sort_values(by ='covid_19_deaths_per_thousand',ascending=True)

corr_deaths

### Now repeat for ONS deaths

In [ ]:
london_corr_input_ons = london_covid_all_ons[['ons_deaths_per_thousand', 'total_population_mid_2018',
                                     'over_70_prop', 'all_bame_prop', 'all_black_prop', 'pakistani_or_bangladeshi_prop',
                                     'all_indian_prop', 'hypertension',
                                     'obesity', 'diabetes', 'asthma', 'heart_disease', 'income_rank', 
                                     'education_rank', 'health_rank', 'housing_rank', 'living_environment_rank']]
london_corr_ons = london_corr_input_ons.corr()

corr_deaths_ons=london_corr_ons[['ons_deaths_per_thousand']]
#sort by the amount of correlation
corr_deaths_ons=corr_deaths_ons.sort_values(by ='ons_deaths_per_thousand',ascending=True)

corr_deaths_ons

In [ ]:
fig = plt.figure(figsize = (20,10))
plt.subplot(1, 2, 1)

sns.heatmap(
    corr_deaths, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)

plt.subplot(1, 2, 2)
sns.heatmap(
    corr_deaths_ons, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)

plt.show()

In [ ]:
# THis is the beginnings of code to create a dataframe with the individual values
# so I can create a bar chart rather than a heatmap (better to compare) - doesn't 
# work yet.

#corr_ons = pd.DataFrame([corr_deaths_ons['ons_deaths_per_thousand'].iloc[0], 
#            corr_deaths_ons['ons_deaths_per_thousand'].iloc[2]])
#            
#corr_ons

#def create_corr_df(label, corr, n):
#    lst = []
#    for x in range(0, n):
#        lst.append((corr[label].iloc[x], ))
#        
#    df = pd.DataFrame(lst)
#    df['name'] = label
#    
#    return df
#        
#corr_ons = create_corr_df('ons_deaths_per_thousand', corr_deaths_ons, 17)
#
#corr_ons



### Conclusions
- Even incorporating death data up until the end of the first wave, it is reasonable to confirm the London Datastore findings that there are no correlations between demographic, health or deprivation factors and the total number of covid deaths in a London boroughs when considered in isolation. So a further avenue of research would be to perform a regression analysis, incrementally adding in each feature to understand which combination of borough characteristics have the biggest correlation with the number of deaths within the borough

### Next steps
The next step is to see whether there was any correlation between neighbouring locations over time between the beginning of March and the end of July. This will show if the virus spread in pockets.

#### Load weekly covid cases by MSOA

In [ ]:
uk_covid_cases = pd.read_excel("MSOAs_latest.xlsx", sheet_name='AmendedData')
print(uk_covid_cases.count())
uk_covid_cases.head()

##### Now we want just msoa for London so get a list and then merge

In [ ]:
london_covid_codes = london_covid_all_ons[['msoa_code', 'msoa_name', 'local_auth_code', 
                                           'local_auth_name', 'covid_19_deaths', 
                                           'ons_total_deaths','total_population_mid_2018']].copy()
london_covid_cases = pd.merge(london_covid_codes, uk_covid_cases, left_on='msoa_code', right_on='msoa_code', how = 'inner')
print(london_covid_cases.count())
london_covid_cases.head()

In [ ]:
london_covid_cases.drop(['msoa_name_y'], axis=1, inplace=True)
london_covid_cases.rename(columns = {'msoa_name_x':'msoa_name'}, inplace = True)
london_covid_cases.head()

In [ ]:
col_names = london_covid_cases.columns.tolist()
col_names

In [ ]:
col_names = london_covid_cases.columns.tolist()
col_names = col_names[7:45]

london_weekly_cases_msoa = pd.melt(london_covid_cases, id_vars=['msoa_code', 
                                                        'msoa_name', 
                                                        'local_auth_code', 
                                                        'local_auth_name',
                                                        'covid_19_deaths',
                                                        'ons_total_deaths',
                                                        'total_population_mid_2018'], 
                        value_vars=col_names, var_name = 'week_number', value_name='cases')

print(london_weekly_cases_msoa.count())
london_weekly_cases_msoa.head()

In [ ]:
temp = london_weekly_cases_msoa.groupby(["msoa_code"]).apply(lambda x: x['cases'][london_weekly_cases_msoa.week_number < 27].sum()).reset_index()
london_weekly_cases_msoa = pd.merge(london_weekly_cases_msoa, temp, left_on='msoa_code', right_on='msoa_code', how = 'inner')
london_weekly_cases_msoa.rename(columns = {0:'cases_to_end_june'}, inplace = True)

london_weekly_cases_msoa.head()

#### Compare deaths to covid cases to see if correlated

In [ ]:
import altair as alt

alt.data_transformers.disable_max_rows()

alt.Chart(london_weekly_cases_msoa).mark_point().encode(
    x='ons_total_deaths:Q',
    y='cases_to_end_june:Q', 
    tooltip = ['msoa_name']
).properties(
    width=180,
    height=180
).interactive()

In [ ]:
import scipy.stats as stats

corrPearson, pValPearson = stats.pearsonr(london_weekly_cases_msoa.ons_total_deaths, london_weekly_cases_msoa.cases_to_end_june)
corrSpearman,pValSpearman = stats.spearmanr(london_weekly_cases_msoa.ons_total_deaths, london_weekly_cases_msoa.cases_to_end_june)

print("Cased versus deaths: Pearson = " + str(corrPearson) + ", Spearman = " + str(corrSpearman) + "," + str(pValPearson))

### Conclusions
There is MODERATE correlation between cases and deaths and so it will be instructive to see how cases by region progress over time. 

First lets load appropriate shape files and merge with our data

In [ ]:
import geopandas as gpd

gb = gpd.read_file("Shapefiles/Middle_Layer_Super_Output_Areas__December_2001__Boundaries_EW_BGC.shp") # a gis format that has geographical boundaries QGIS is a package for looking at shape files
gb.crs = "epsg:27700" # code for the UK national grid

In [ ]:
gb.head()

In [ ]:
london_covid_geo = pd.merge(gb, london_weekly_cases_msoa, left_on='MSOA01CD', right_on='msoa_code', how = 'inner')

In [ ]:
data_geo = alt.InlineData(values = london_covid_geo.to_json(), #geopandas to geojson string
                       format = alt.DataFormat(property='features',type='json'))

alt.Chart(data_geo).mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
).encode(
    color=alt.Color('properties.ons_total_deaths:Q'),
    tooltip=['properties.msoa_name:N', 'properties.ons_total_deaths:Q']
).properties(
    projection={'type': 'identity','reflectY': True},
    width=800,
    height=1200,
    title='deaths by msoa - to jul'
)

### Discussion - we can see the pockets of London having with the highest concentration of covid deaths by July but now I want to see cases by week in a waffle chart

In [ ]:
london_covid_temp = london_covid_all_ons[['msoa_code', 'msoa_name', 'local_auth_code', 'local_auth_name', 
                                          'covid_19_deaths', 'covid_19_deaths_per_thousand', 'ons_total_deaths',
                                          'ons_deaths_per_thousand', 'march_deaths', 'april_deaths', 'may_deaths',
                                          'june_deaths', 'july_deaths', 'total_population_mid_2018']].copy()

london_cases_temp = london_covid_cases[['msoa_code', 10, 11, 12, 13, 14, 15,
                                          16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]].copy()

london_coords_temp = pd.merge(london_covid_temp, london_cases_temp, 
                              left_on='msoa_code', right_on='msoa_code', how = 'inner')

print(london_coords_temp.count())
london_coords_temp.head()

In [ ]:
soa_shape_map = gpd.read_file('Shapefiles/Middle_Layer_Super_Output_Areas__December_2001__Boundaries_EW_BGC.shp')

soa_shape_map_geo = soa_shape_map.to_crs(epsg=4326)

soa_shape_map_geo['long'] = soa_shape_map_geo.geometry.centroid.x
soa_shape_map_geo['lat'] = soa_shape_map_geo.geometry.centroid.y

london_covid_shapes = pd.merge(soa_shape_map_geo, london_coords_temp, left_on='MSOA01CD', right_on='msoa_code', how = 'inner')

print(london_covid_shapes.count())
london_covid_shapes

In [ ]:
london_covid_shapes.to_csv('output_data/london_covid_shapes.csv', index=False)

In [ ]:
print("max-lon=" + str(london_covid_shapes.long.max()))
print("min-lon=" + str(london_covid_shapes.long.min()))
print("max-lat=" + str(london_covid_shapes.lat.max()))
print("min-lat=" + str(london_covid_shapes.lat.min()))

In [ ]:
lon_range = (-0.5, 0.35)
lon_cells = 15

lat_range = (51.2, 51.8)
lat_cells = 13

In [ ]:
from shapely.geometry import Polygon

lon_incr = (lon_range[1] - lon_range[0]) / lon_cells
lat_incr = (lat_range[1] - lat_range[0]) / lat_cells
x0, y0 = lon_range[0], lat_range[0]

cell_ids = []
grid_cells = []
for c in range(lon_cells):
    x1 = x0 + lon_incr
    for r in range(lat_cells):
        y1 = y0 + lat_incr
        grid_cells.append(Polygon([(x0,y0),(x0,y1),(x1,y1),(x1,y0)]))
        cell_ids.append('{:02d}_{:02d}'.format(c, r))
        y0 = y1
    x0 = x1
    y0 = lat_range[0]


In [ ]:
london_grid_temp = pd.melt(london_covid_shapes, id_vars=['msoa_code', 
                                                        'long', 
                                                        'lat'], 
                        value_vars=[10, 11, 12, 13, 14, 15, 16, 17, 18, 
                                    19, 20, 21, 22, 23, 24, 25, 26], var_name = 'week', value_name='cases')

print(london_grid_temp.count())
london_grid_temp.head()

In [ ]:
london_grid_temp['grid_x']   = np.floor((london_grid_temp['long'] - lon_range[0]) / (lon_range[1] - lon_range[0]) * lon_cells).astype(int)
london_grid_temp['grid_y']   = np.floor((london_grid_temp['lat']  - lat_range[0]) / (lat_range[1] - lat_range[0]) * lat_cells).astype(int)
# The cell_id column will be used to link our aggregate data to the grid GeoJSON object for plotting
london_grid_temp['cell_id']  = london_grid_temp[['grid_x','grid_y']].apply(lambda x: '{:02d}_{:02d}'.format(x.grid_x, x.grid_y), axis=1)
london_grid_temp['interval'] = london_grid_temp['week']

london_grid_temp.head()

In [ ]:
london_grid = london_grid_temp.groupby(['cell_id', 'grid_x', 'grid_y', 'week']).sum().reset_index()
london_grid['cummulative_deaths'] = london_grid.groupby(['cell_id', 'grid_x', 'grid_y'])['cases'].cumsum()

london_grid.to_csv('output_data/london_grid.csv', index=False)
london_grid.head()

In [ ]:
alt.Chart(london_grid).mark_square().encode(
    x='grid_x:O',
    y='grid_y:O',
    color = alt.Color('cummulative_deaths:Q', scale=alt.Scale(scheme='reds')), 
    tooltip = ['cummulative_deaths']
).properties(
    width=180,
    height=180
).facet(
    column='week:O'
).interactive()